# Přednáška: Zobecněné lineární modely (GLM, Generalized Linear Models)

**Obsah přednášky**
- Motivační příklad
- GLM - základní principy
- logistická regrese
- sigmoid fce, logistická fce (scipy)
- kvantifikace chyb
- příklad

## Motivační příklady

Klasické lineární modely mají těžkosti v případech, kdy modelovaná veličina vykazuje nějaká omezení, např. je z intervalu [0, 1], dichotomická {0, 1} resp. {ano, ne}, nebo jde o počet. Některé případy:

- Počet zákazníků eshopu v závislosti na sezoně, slevové akci, způsobu dopravy, hodnocení a druhu reklamy.
- Výsledek léčby zhoubného onemocnění v závislosti na podávané medikaci a predispozic pacienta.
- Výsledek přijímacího řízení (přijat-nepřijat) na univerzitu v závislosti na předchozích studijních výsledcích a jiných faktorech.
- Vypuknutí či nevypuknutí infekce po císařském řezu (SC) v závislosti na tom, zda SC byl plánovaný či nikoliv, přítomnosti rizikových faktorů (diabetes, nadváha), profylaxe ATB.
- Počet úmrtí v důsledku onemocnění dýchacích cest v závislosti na úrovni koncentrace polutantů v ovzduší.
- Počet vysokoenergetických částic dopadnuvších na detektor družice v závislosti na sluneční aktivitě, přítomnosti/třídě sluneční erupce, poloze družice.

## GLM - trocha matematiky 

Označme stejně jako v případě dříve probraných **lineárních modelů** pozorovanou náhodnou veličinu $y_t, t=1,2,\ldots$. Budeme předpokládat, že má nějaké rozdělení z **exponenciální třídy distribucí**, což jak už rovněž víme, může být například normální, binomické, Poissonovo nebo gama. Stejně jako u lineárních modelů budeme uvažovat **vysvětlující (nezávislou) proměnnou - regresor** $x_t$ s realizacemi $x_1, x_2,\ldots$ a dále **vektor regresních koeficientů** $\beta$ o stejném rozměru jako $x_t$.

Vzpomeňme na lineární regresi, kde $y_t = \beta^\intercal x_t + \varepsilon_t$.

> **Zobecněné lineární modely** jsou modely ve tvaru
>
>$$
\widehat{y}_t = \mathbb{E}[y_t|x_t, \beta] = g^{-1}(\beta^\intercal x_t),
$$
>
> kde $\widehat{y}_t = \mathbb{E}[y_t|x_t, \beta]$ je střední hodnota $y_t$, $\beta^\intercal x_t$ je lineární prediktor stejně jako v lineární regresi a $g$ je linková (spojovací) funkce.

Ještě lépe pochopitelné to bude po zmínce, že střední hodnota se někdy nazývá *očekávanou hodnotou*, angl. expected value nebo expectation.

Z rovnice výše plyne, že
$$
\beta^\intercal x_t = g(\mathbb{E}[y_t|x_t, \beta]) = g(\widehat{y}_t),
$$

tedy že linková funkce spojuje lineární prediktor $\beta^\intercal x_t$ se střední (očekávanou) hodnotou $y_t$.

**Úkol:**
- lineární regrese je rovněž zobecněným lineárním modelem. Jak v tom případě vypadají funkce $g$ a $g^{-1}$?

### Příklady linkových funkcí $g$
#### Dichotomická veličina $y \sim \{0, 1\}$
Pro binární (dichotomickou) veličinu s *Bernoulliho rozdělením* s pravděpodobností $p$ výhodou používáme funkci [logit](https://en.wikipedia.org/wiki/Logit):

$$
\mathit{logit}(p) = \log\left(\frac{p}{1-p}\right).
$$

Její inverze se nazývá [logistickou funkcí](https://en.wikipedia.org/wiki/Logistic_function), též sigmoid funkcí (to je ale spíše [celá množina funkcí](https://en.wikipedia.org/wiki/Logit)). Logistická funkce má tvar

$$
\mathit{logit}^{-1}(z) = \sigma(z) = \frac{e^z}{1+e^z} = \frac{1}{1+e^{-z}}.
$$

Za argument $z$ dosazujeme $\beta^\intercal x_t$, čímž dostáváme **logistickou regresi**. Grafy obou funkcí ukazují obrázky.

![logit](img/logit_expit.png)

#### Poissonovská veličina $y \sim \textit{Poisson}(\cdot)$
Poissonovo rozdělení používáme pro popis počtu jevů nastalých v určitém místě nebo časovém okamžiku, např. počet částic dopadnuvších na detektor za daný časový okamžik, počet zákazníků kontaktujících call centrum v závislosti na velikosti lokality, počet úmrtí v dané věkové kategorii za definovaný časový úsek aj. Jelikož potřebujeme lineární prediktor $\beta^\intercal x_t$ zobrazit nějakým způsobem na množinu kladných čísel, můžeme s výhodou využít logaritmus. Inverzní zobrazení - počet realizací - nám pak dodá exponenciální funkce.

Jinými slovy: **Poissonovská regrese** pracuje s modelem

$$
\log \mathbb{E}[y|x, \beta] = \beta^\intercal x, \qquad \text{respektive} \qquad \mathbb{E}[y|x, \beta] = e^{\beta^\intercal x}
$$

## Logistická regrese

Logistický regresní model je určen pro popis dichotomické veličiny

$$
y_t = 
\begin{cases} 
1 \quad \text{s pravděpodobností $p_t$}, \\ 
0 \quad \text{s pravděpodobností $1 - p_t$}.\\
\end{cases}
$$

Z dřívějška víme, že vhodnou distribucí pro $y_t$ je distribuce **Bernoulliho** (též alternativní) s pravděpodobností $p_t$ jako svým parametrem,

$$
y_t \sim \textit{Bernoulli}(p_t)
$$

a **se střední hodnotou** $\mathbb{E}[y_t] = p_t$.

Máme-li k dispozici nezávislou vysvětlující proměnnou (regresor) $x_t$, pokusíme se najít vhodné $\beta$ a $y_t$ s nimi vhodně provázat.

Připomeňme ještě, že pravděpodobnostní funkce Bernoulliho rozdělení je

$$
\begin{aligned}
f(y_t|p_t) &= p_t^{y_t} (1-p_t)^{1-y_t} \\
           &= f(y_t|x_t, \beta),
\end{aligned}
$$

kde druhý řádek jsme si doplnili pro pořádek, důvod se ukáže vzápětí :-)

Z teorie GLM výše víme, že potřebujeme vhodnou linkovou funkci, která prováže *lineární prediktor* $\beta^\intercal x_t$ s $y_t$, respektive jeho střední (očekávanou) hodnotou,

$$
\widehat{y}_t = \mathbb{E}[y_t|x_t, \beta] = g^{-1}(\beta^\intercal x_t).
$$

A dokonce už také víme, že vhodnou volbou pro $g$ je funkce $\textit{logit}$, a tedy

$$
\begin{aligned}
g(\mathbb{E}[y_t|p_t]) 
&= g(\mathbb{E}[y_t|X_t, \beta]) \\
&= g(p_t) \\
&= \textit{logit}(p_t) \\
&= \log \frac{p_t}{1-p_t}
= \beta^\intercal x_t. 
\end{aligned}
$$

Analogicky tedy inverzí
$$
\begin{aligned}
\widehat{y}_t = \mathbb{E}[y_t|x_t, \beta]
&= p_t \\
&= \textit{logit}^{-1}(\beta^\intercal x_t) \\
&= \sigma(\beta^\intercal x_t) \\
&= \frac{1}{1+e^{-\beta^\intercal x_t}}.
\end{aligned}
$$

### Bayesovský odhad logistického regresního modelu

Bayesovský odhad $\beta$ není triviální, neboť aposteriorní distribuce

$$
\begin{aligned}
\pi(\beta|x_{0:t}, y_{0:t}) 
&\propto f(y_t|x_t, \beta) \times \pi(\beta|x_{0:t-1}, y_{0:t-1}) \\
&\propto p_t^{y_t} (1-p_t)^{1-y_t}) \times \pi(\beta|x_{0:t-1}, y_{0:t-1}) \\
&\propto \widehat{y}_t^{y_t}
(1-\widehat{y}_t)^{1-y_t} \times \pi(\beta|x_{0:t-1}, y_{0:t-1})
\end{aligned}
$$

není analyticky dostupná pro žádnou rozumnou volbu apriorna. Možností, jak k problému přistoupit je povícero - buď odhad pomocí metod Monte Carlo, nebo variačními metodami, nebo aproximativně laplaceovskou aproximací. My si vybereme tu poslední variantu.

Zvolíme si jako apriorno vícerozměrnou normální distribuci

$$
\beta|x_{0:t-1}, y_{0:t-1} \sim \mathcal{N}(\widehat{\beta}_{t-1}, \Sigma_{t-1}). 
$$

Aposteriorní distribuci pořešíme aproximativně ve dvou krocích:
1. Najdeme maximum aposteriorní distribuce uvedené výše. Tím dostaneme MAP (maximum a posteriori) odhad.
2. Do tohoto maxima umístíme střední hodnotu normální hustoty a najdeme pro ni odpovídající vhodnou kovarianci.

Na následujícím obrázku vidíme jednotlivé kroky. Nejprve máme nějakou nestandardní distribuci (vlevo), nalezneme její maximum (uprostřed) a do něj umístíme střední hodnotu aproximující normální distribuce. Její varianci roztáhneme tak, aby rozumně pokrývala původní hustotu (vpravo). Samozřejmě, že laplaceovská aproximace není dokonalá a čím "ošklivější" aproximovaná hustota, tím horší je kvalita aproximace.

![laplace](img/laplace.png)



**Úkoly:**
- obecně: jakým způsobem hledáme maximum funkce?

> **Intermezzo: Newtonova metoda**
>
> Je-li dána dostatečně "rozumná" funkce $g$ - zejm. spojitá s existující derivací a monotónní v určeném intervalu, pak pro nalezení bodu $x$ v němž platí  $g(x)=0$ lze použít Newtonovu metodu. Odstartujeme-li z "rozumného" bodu $x_0$, najdeme nový bod $x_1$ na průsečíku tečny v $g(x_0)$ s osou $x$ a takto postupně iterujeme až do dosažení požadované přesnosti nebo počtu iterací. Ve třech krocích nám nalezení $x_1$ vysvětlí následující obrázek.
![Newtonova metoda](img/Newton2.png)

**Úkoly:**
- Už víme, že budeme hledat maximum aposteriorní distribuce $\pi(\beta|x_{0:t}, y_{0:t})$. Co bude analogií k funkci $g$ v "Intermezzu"?

#### Hledání střední hodnoty aproximující normální distribuce

Už tedy víme, že budeme "newtonovsky" hledat maximum $\widehat{\beta}_t$. Můžeme to dělat i jen jednokrokovým Newtonem, kde za počáteční hodnotu (analogické k $x_0$) vybereme minulý odhad $\widehat{\beta}_{t-1}$,

$$
\widehat{\beta}_t = \widehat{\beta}_{t-1} - [l''(\widehat{\beta}_{t-1})]^{-1} l'(\widehat{\beta}_{t-1}),
$$

kde

$$
l(\widehat{\beta}_{t-1}) = \log \pi(\beta|x_{0:t}, y_{0:t}) 
$$

je logaritmus aposteriorní hustoty v bodě $\widehat{\beta}_{t-1}$. Pro úplnost dodejme, že

$$
\begin{aligned}
l'(\widehat{\beta}_{t-1}) 
&= \left(y_t - \widehat{y}_t \right)x_t,\\
l''(\widehat{\beta}_{t-1})
&= -\Sigma_{t-1} - \widehat{y}_t(1-\widehat{y}_t) x_t x_t^\intercal.
\end{aligned}
$$

#### Hledání kovariance aproximující normální hustoty
Pokud jsme nalezli MAP odhad, nafitujeme normální distribuci s kovariancí

$$
\Sigma_t = -[l''(\widehat{\beta}_{t-1})]^{-1}.
$$

To již máme spočteno z předchozího kroku.

### Algoritmus

Algoritmus tedy vypadá následovně:

**Inicializace**
- Nastavíme vhodné počáteční hyperparametry apriorního normálního rozdělení $\widehat\beta_{t-1}, \Sigma_{t-1}$.

**Online modelování**
1. Získáme nová měření $y_t$ a regresor $x_t$.
2. Napočítáme MAP odhad $\widehat{\beta}_t$.
3. Napočítáme aproximativní kovarianci odhadu $\Sigma_t$.

### Predikce

Bayesovská predikce běží přes prediktivní distribuci, která ovšem rovněž není analyticky dostupná. Místo toho můžeme využít laplaceovskou aproximaci

$$
\begin{aligned}
f(y'|x', x_{0:t}, y_{0:t}) 
&= \int f(y'|x', x_{0:t}, y_{0:t}, \beta) \pi(\beta|x_{0:t}, y_{0:t}) d\beta  \\
&\approx
(2\pi)^{\frac{n}{2}} \left[\det l''(\widehat{\beta}_t)\right]^{-\frac{1}{2}} f(y'|x', \beta)\ \pi(\beta|x_{0:t},y_{0:t}),
\end{aligned}
$$

kde dosadíme $\beta = \widehat{\beta}_t$ a $n$ je rozměr $\beta$.

#### Predikce - dodatek
Stojí za zmínku, že ať už k predikci přistoupíme bayesovsky nebo ne, budeme dostávat reálné číslo od 0 do 1. Jenže naše predikovaná veličina je *dichotomická*,  *binární*, tedy nabývá hodnot 0 nebo 1. Pro účely predikce tedy zpravidla stanovíme mez $m \in [0, 1]$, většinou 0.5, a predikujeme
$$
\widehat{y} =
\begin{cases}
0 \Leftrightarrow y' < m, \\
1 \Leftrightarrow y' \geq m.
\end{cases}
$$

## Diagnostika výsledků (kvality predikce)

### Brierovo skóre
Pro $T$ predikcí $y_t'$ skutečně pozorované veličiny $y_t$, kde $y_t'$ může být jak z intervalu [0, 1], tak nabývat hodnot z {0, 1} - což je nutné specifikovat - definujeme [Brierovo skóre](https://en.wikipedia.org/wiki/Brier_score) jako průměrnou kvadratickou chybu:
$$
B = \frac{1}{T} \sum_{t=1}^{T} (y_t' - y_t)^2.
$$

Brierovo skóre je ekvivalentem střední kvadratické chyby predikce, [MSE](https://en.wikipedia.org/wiki/Mean_squared_error).


### Matice záměn (Confusion matrix (table))
[Matice záměn](https://en.wikipedia.org/wiki/Confusion_matrix#Table_of_confusion) je typ [kontingenční tabulky](https://en.wikipedia.org/wiki/Contingency_table) pro úlohy klasifikace. Zanamenává vztah mezi skutečnou veličinou a její predikcí, např. klasifikujeme-li spam, pak tato matice vypadá následovně:

| | Spam | Non-Spam |
| --- | --- | --- |
|**Pred. spam** | 100 | 2 |
|**Pred. non-spam** | 10 | 500 |

Na matici lze napočítat celou řadu důležitých i méně důležitých ukazatelů kvality predikce. Následující tabulka ukazuje řadu z nich. Červeně orámována jsou data která jsou v příkladu výše, okolo jsou pak ony ukazatele. Mezi nejvýznamnější patří hlavně [senzitivita a specificita](https://en.wikipedia.org/wiki/Sensitivity_and_specificity), v aplikované statistice dále [síla testu](https://en.wikipedia.org/wiki/Statistical_power), [chyba prvního a druhého druhu](https://en.wikipedia.org/wiki/Type_I_and_type_II_errors), [DOR](https://en.wikipedia.org/wiki/Diagnostic_odds_ratio) a některé jiné.
![confusion_matrix](img/confusion_matrix.png)
Zdroj: [https://en.wikipedia.org/wiki/Confusion_matrix#Table_of_confusion]

# Klasifikace Skin - NonSkin

Praktickou ukázkou použití logistické regrese (nebo libovolné jiné vhodné klasifikační metody) je určování, zda daný pixel získaný z fotografie obličeje patří ke kůži (Skin) či nikoliv (NonSkin). Pro testování algorimů se využívá databáze *Skin Segmentation Dataset* (Bhatt & Dhall) získaná náhodným vzorkováním RGB (Red-Green-Blue) hodnot pixelů z obličejů různých věkových skupin (young-middle-old) a rasových skupin (white-black-asian) z databází FERET a PAL. Dataset obsahuje celkem 245 057 řádků s hodnotami B, G, R a příslušností k třídě Skin (1) resp. NonSkin (2). Z toho je 194 198 vzorků NonSkin a 50 859 Skin. V tomto příkladu použijeme 1000 náhodně vybraných vzorků a sekvenčně je zkusíme oklasifikovat logistickou regresí. Použijeme regresor ve tvaru $x_t = [1, B_t, G_t, R_t]$, normální apriorní distribuci $\mathcal{N}$(**_0_**, 100**_I_**) a mez _m_=0.5.

Tři náhodně vybrané datové řádky:

    ---
    B, G, R, Class
    242, 169, 161,   2
    218, 211, 202,   2
    110, 150, 209,   1
    ---

## Výsledky:

| | Skin | NonSkin | Sum |
|---|---|---|---|
|Pred. Skin| 188 | 42 | 230 |
|Pred. NonSkin| 51 | 719 | 770 |
| Sum | 239 | 761 | 1000|


    TPR: 0.933766233766
    TNR: 0.817391304348
    PPV: 0.944809461235
    NPV: 0.786610878661
    FPR: 0.182608695652
    FDR: 0.0551905387648
    FNR: 0.0662337662338
    ACC: 0.907
    F1_score: 0.939255388635
    informedness = TPR+TNR-1: 0.751157538114
    markedness = PPV+NPV-1: 0.731420339896
    prevalence: 0.77
    LRP: 5.11348175634
    LRN: 0.0810306714562
    DOR: 63.1055088702
    FOR: 0.213389121339


Vývoj Brierova skóre ukazuje následující obrázek (finální hodnota 0.085):
![Brier](img/l5-brier.png)